In [1]:
import requests
import json
from datetime import datetime
import time
import geocoder
import geopy
import math
from geopy.geocoders import Nominatim
import haversine 
from haversine import Unit


* I'm using Jupyter Notebook to develop the classes because it's simple like a sandbox.

In [2]:
class Iss():

    def __init__ (self):

        self.welcome = "Ladies and Gentlemen, welcome aboard the ISS!"
        self.response = requests.get("http://api.open-notify.org/iss-now.json").json() 
        self.iss_time = datetime.fromtimestamp(self.response['timestamp']).strftime('%H:%M:%S')


    def verify(self):
        return "Sucess! We're floating in space!" if self.response['message'] == 'success' else "Ladies and Gentlemen, we're out of orbit!"

    def whos_aboard (self):
        return [names['name'] for names in requests.get("http://api.open-notify.org/astros.json").json()['people'] if names['craft'] == 'ISS']

    def position (self):
        self.iss_latitude = float(self.response['iss_position']['latitude'])
        self.iss_longitude = float(self.response['iss_position']['longitude'])

        return self.iss_latitude , self.iss_longitude 

In [3]:
class Where_am_I():

    def __init__(self, nome=None):
        self.nome = nome
        self.current_time = datetime.now().strftime('%H:%M:%S')

    def current_location_name (self):
        return geocoder.ip('me')
    
    def coordinates(self):
        self.latitude = geocoder.ip('me').lat
        self.longitude = geocoder.ip('me').lng

        return self.latitude , self.longitude

In [4]:
class Astrolabe(Iss, Where_am_I):
    
    def __init__(self):

        super().__init__()


    def distance_km(self):   

       return round((haversine.haversine(self.position(),self.coordinates(), unit = Unit.KILOMETERS)),2)


    def match(self):

        if self.distance_km() <= 100:
            return "Are you aboard?"
        elif self.distance_km() <= 250:
            return "The ISS is really closer you!"
        elif self.distance_km() <= 500:
            return "The ISS is approaching!"
        elif self.distance_km() <= 650:
            return "The ISS is approaching!"
        elif self.distance_km() <= 1000:
            return "I think I saw something in the sky"
        else:
            return "The ISS is so far..."


    def compass_ISS(self):
        
        self.lat_iss =  self.long_iss = None

        self.lat_iss = "North" if self.position()[0] > 0 else "South"
        self.long_iss =  "East" if self.position()[1] > 0 else "West"

        return f"ISS - Lat_issitude: {self.lat_iss} and Longitude: {self.long_iss}"

    def iss_complete_location(self):
        
        try:
            self.geolocation = Nominatim(user_agent="geoapiExercises")
            self.location = self.geolocation.reverse(str(self.position()[0])+","+str(self.position()[1])).raw['address']
            return self.location     

        except:
            return "Did the ISS enter a black hole?"
        
    def iss_country(self):

        self.contry_loc = None

        self.country_loc = Astrolabe().iss_complete_location()['country'] if 'country' in self.iss_complete_location() else "--"

        return self.country_loc


In [5]:
from PIL import Image
from io import BytesIO

class ImageOfDay():

    def __init__ (self):
        self.about = "Welcome to the image of Day, by NASA."
        self.request = None

    def validate_key(self):
        self.answer = None
        self.key_nasa  = None
        
        negative,positive = ["No", "no", "N", "n"],["Yes", "yes", "Y", "y"]

        while self.answer not in negative or positive:
            self.answer = input("Do you have a Key? (Y/N)")

            if self.answer in negative:
                self.request = requests.get(f"https://api.nasa.gov/planetary/apod?api_key=DEMO_KEY").json()
                print("Now you're using the default Key (DEMO_KEY): \nHourly Limit: 30 requests per IP address per hour. \nDaily Limit: 50 requests per IP address per day.")
                break

            elif self.answer in positive:
                self.key_nasa = input("Please, enter the key:")
                self.request = requests.get(f"https://api.nasa.gov/planetary/apod?api_key={self.key_nasa}").json()
                break
            
            print("Please, answer correctly, Cosmonaut!")

    def get_image(self):
        if self.request != None:
            return  requests.get(self.request['hdurl']), Image.open(BytesIO(requests.get(self.request['hdurl']).content)).show()
        else:
            print("Please, validate the key:")
            return self.validate_key(), requests.get(self.request['hdurl']), Image.open(BytesIO(requests.get(self.request['hdurl']).content)).show()
        
    def get_explanation(self):
        if self.request != None:
            return  self.request['title'], self.request['explanation']
        else:
            print("Please, validate the key:")
            return self.validate_key(), self.request['title'], self.request['explanation']
        
        

In [15]:
import psycopg2


class ISS_Monitoring(Astrolabe, Iss):

    def __init__(self):

        super().__init__()

        self.user = None
        self.password = None
        self.host = None
        self.port = None
        self.database = None
        self.connection = None


    def iss_database_connection(self):

        while True:

            self.user = input("Database user:")
            self.password = input("Database password:")
            self.host = input("Database host:")
            self.port = input("Database port:")
            self.database = input("Database:")

            try:

                self.connection = psycopg2.connect(
                user = self.user,
                password = self.password,
                host = self.host,
                port = self.port,
                database = self.database
                )

                return "Success!"
    
            except psycopg2.Error as error:
    
                 print(f"Database connection error! \n-- {error}\nPlease try again!") 

            except KeyboardInterrupt:

                print("Interrupted")   



    def iss_data_streaming(self):
        
        self.psql_query = """INSERT INTO iss_monitoring ("iss_lat", "iss_long", "current_country", "iss_time", "current_time", "current_date") VALUES (%s, %s, %s, %s, %s, %s);"""
        self.warning = "Connection database OK!\nStarting the streaming... \nPress CTRL+C to stop."
        self.data = None
        self.data_err = None
        self.cursor = None


        try:

            self.iss_database_connection() if self.connection == None else print(self.warning)

            print(self.warning)

            self.cursor = self.connection.cursor()
            
            while True:


                if Iss().response['message'] == 'success':
            
                    self.data = (Iss().position()[0],
                        Iss().position()[1],
                        Astrolabe().iss_country(),
                        Iss().iss_time,
                        datetime.now().strftime('%H:%M:%S'),
                        datetime.now().strftime('%Y-%m-%d')
                    )
            
                    self.cursor.execute(self.psql_query, self.data)
            
                    self.connection.commit()
            
                    time.sleep(30)

                else:

                    self.data_err = (
                        None,
                        None,
                        "COMMUNICATION ERROR",
                        None,
                        datetime.now().strftime('%H:%M:%S'),
                        datetime.now().strftime('%Y-%m-%d')
                    )
            
                    self.cursor.execute(self.psql_query, self.data_err)

                    self.connection.commit()

                    print("ISS Communication Error")

                    time.sleep(30)


        except KeyboardInterrupt:

            if self.connection:
                self.cursor.close()
                self.connection.close()
                print ("PostgreSQL connection is closed")
            pass

    